## Question 2: CRUD Operations in SQL Server

In [1]:
# Import Python packages
import numpy as np
import pandas as pd
from datetime import date, datetime

Create an SQL table schema to store time-series metal prices. Include fields like Date, Metal, Price. 
I think we are not meant to populate the table with the contents of the CSV file so skip the next 3 cells.

In [2]:
# data_file = "../data/MarketData.csv"
# try:
#     # Read in data set from the csv file, skip initial metadata rows (first 3 rows)
#     f = open(data_file, 'rb')
#     df = pd.read_csv(f, skiprows = 3, header = [0, 1, 2, 3])

#     # Drop the first column name from the multi-index (for "Dates" column)
#     df.columns = ['Dates'] + [f"{col[0]} ({col[1]})" for col in df.columns[1:]]
    
# except:
#     print('Error opening file/loading data')

In [3]:
# Convert "Dates" column to datetime format, European date format
# df["Dates"] = pd.to_datetime(df["Dates"], dayfirst=True)

# # Display the first few rows
# df

,Dates,LME COPPER 3MO ($) (LMCADS03 Comdty),LME ALUMINUM 3MO ($) (LMAHDS03 Comdty),LME ZINC 3MO ($) (LMZSDS03 Comdty),LME LEAD 3MO ($) (LMPBDS03 Comdty),LME TIN 3MO ($) (LMSNDS03 Comdty),Generic 1st 'CL' Future (CL1 Comdty)
0,2010-01-01,7375.0,2230.0,2560.0,2432.0,16950,79.36
1,2010-01-04,7500.0,2267.0,2574.0,2515.0,17450,81.51
2,2010-01-05,7485.0,2302.0,2575.0,2522.5,17375,81.77
3,2010-01-06,7660.0,2377.0,2718.0,2680.0,17825,83.18
4,2010-01-07,7535.0,2310.0,2607.0,2599.0,17475,82.66
...,...,...,...,...,...,...,...
3386,2022-12-26,8349.5,2389.5,2965.0,2273.5,23934,79.56
3387,2022-12-27,8349.5,2389.5,2965.0,2273.5,23934,79.53
3388,2022-12-28,8443.0,2381.0,3005.5,2218.0,24734,78.96
3389,2022-12-29,8418.0,2405.0,2984.5,2272.5,24915,78.40


The data has been read in "wide format". According to the instructions, we'd like to save it in "long format" in the SQL database, therefore me need to transform the pandas dataframe to long format


In [4]:
# Use melt() to transform from wide to long format
# df_long = df.melt(id_vars=['Dates'], var_name='Metal', value_name='Price')
# df_long

,Dates,Metal,Price
0,2010-01-01,LME COPPER 3MO ($) (LMCADS03 Comdty),7375.00
1,2010-01-04,LME COPPER 3MO ($) (LMCADS03 Comdty),7500.00
2,2010-01-05,LME COPPER 3MO ($) (LMCADS03 Comdty),7485.00
3,2010-01-06,LME COPPER 3MO ($) (LMCADS03 Comdty),7660.00
4,2010-01-07,LME COPPER 3MO ($) (LMCADS03 Comdty),7535.00
...,...,...,...
20341,2022-12-26,Generic 1st 'CL' Future (CL1 Comdty),79.56
20342,2022-12-27,Generic 1st 'CL' Future (CL1 Comdty),79.53
20343,2022-12-28,Generic 1st 'CL' Future (CL1 Comdty),78.96
20344,2022-12-29,Generic 1st 'CL' Future (CL1 Comdty),78.40


In [2]:
# Import sqlalchemy packages
from sqlalchemy import create_engine, inspect, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import create_engine


In [3]:
# Define the database URI
engine = create_engine('sqlite:///metal_prices.db')

Define SQL Table Schema

Connect to SQL Database Using SQLAlchemy

In [4]:
# Define ORM model to easily access the data
Base = declarative_base()

class MetalPrice(Base):
    __tablename__ = 'metal_prices'

    Dates = Column(DateTime, primary_key=True)
    Metal = Column(String, primary_key=True)
    Price = Column(Float)

Base.metadata.create_all(engine)

I think I am not meant to populate the table at this stage so please skip this cell!

In [15]:
# Write to database
# df_long.to_sql('metal_prices', con=engine, if_exists='replace', index=False)

20346

Perform CRUD Operations in SQLAlchemy. First query data from the database

In [5]:
# Create session factory
Session = sessionmaker(bind=engine)

Print out first 10 rows of the dataframe from the database.

In [6]:
with Session() as session:

    results = session.query(MetalPrice).all()

    # Print each row
    for row in results[:10]:
        print(f"Dates: {row.Dates}, Metal: {row.Metal}, Price: {row.Price}")

Create (Insert New Data)

In [7]:
# Create
with Session() as session:
    obj = MetalPrice(Dates=date(2020,1,1), Metal='Gold', Price=1000)
    session.add(obj)
    session.commit()

In [8]:
with Session() as session:

    results = session.query(MetalPrice).all()

    # Print each row
    for row in results[-10:]:
        print(f"Dates: {row.Dates}, Metal: {row.Metal}, Price: {row.Price}")

Dates: 2020-01-01 00:00:00, Metal: Gold, Price: 1000.0


In [9]:
# Read
with Session() as session:
    obj = session.query(MetalPrice).filter(MetalPrice.Metal == "Gold").first()
    print(obj.Dates, obj.Metal, obj.Price)

2020-01-01 00:00:00 Gold 1000.0


Update Data

In [10]:
# Update
with Session() as session:
    obj = session.query(MetalPrice).filter(MetalPrice.Metal == "Gold").first()
    obj.Price=2000
    session.commit()

In [11]:
# Read
with Session() as session:
    obj = session.query(MetalPrice).filter(MetalPrice.Metal == "Gold").first()
    print(obj.Dates, obj.Metal, obj.Price)

2020-01-01 00:00:00 Gold 2000.0


Delete Data

In [12]:
# Delete
with Session() as session:
    obj = session.query(MetalPrice).filter(MetalPrice.Metal == "Gold").first()
    session.delete(obj)
    session.commit()

Let's check if gold is still there:

In [13]:
with Session() as session:

    results = session.query(MetalPrice).all()

    # Print each row
    for row in results:
        print(f"Dates: {row.Dates}, Metal: {row.Metal}, Price: {row.Price}")

Originally, I populated the database with the content of the CSV file, therefore I'll migrate the contents and empty the original table

In [20]:
from sqlalchemy import MetaData

# Drop the existing table
meta = MetaData()
meta.reflect(bind=engine)
metal_prices_table = meta.tables['metal_prices']
metal_prices_table.drop(bind=engine)

# Recreate the table with the updated model
Base.metadata.create_all(engine)